In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from bokeh.plotting import output_file, show
from bokeh.io import curdoc
from lacerta.heatmap_scatter import heatmap_scatter

import streamlit as st
from bokeh.embed import file_html
from bokeh.resources import CDN
import base64


output_file('heatmap.html')

INFO:bokeh.io.state:Session output file 'heatmap.html' already exists, will be overwritten.


In [2]:
data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/slump/slump_test.data")
data

,No,Cement,Slag,Fly ash,Water,SP,Coarse Aggr.,Fine Aggr.,SLUMP(cm),FLOW(cm),Compressive Strength (28-day)(Mpa)
0,1,273.0,82.0,105.0,210.0,9.0,904.0,680.0,23.0,62.0,34.99
1,2,163.0,149.0,191.0,180.0,12.0,843.0,746.0,0.0,20.0,41.14
2,3,162.0,148.0,191.0,179.0,16.0,840.0,743.0,1.0,20.0,41.81
3,4,162.0,148.0,190.0,179.0,19.0,838.0,741.0,3.0,21.5,42.08
4,5,154.0,112.0,144.0,220.0,10.0,923.0,658.0,20.0,64.0,26.82
...,...,...,...,...,...,...,...,...,...,...,...
98,99,248.3,101.0,239.1,168.9,7.7,954.2,640.6,0.0,20.0,49.97
99,100,248.0,101.0,239.9,169.1,7.7,949.9,644.1,2.0,20.0,50.23
100,101,258.8,88.0,239.6,175.3,7.6,938.9,646.0,0.0,20.0,50.50
101,102,297.1,40.9,239.9,194.0,7.5,908.9,651.8,27.5,67.0,49.17


In [5]:
# curdoc().theme = "night_sky"
layout = heatmap_scatter(data, theme="night_sky")
show(layout)

In [4]:
html = file_html(layout, CDN, "Correlation Heatmap")


'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="utf-8">\n    <title>Correlation Heatmap</title>\n    <style>\n      html, body {\n        box-sizing: border-box;\n        height: 100%;\n        margin: 0;\n        padding: 0;\n      }\n    </style>\n    <script type="text/javascript" src="https://cdn.bokeh.org/bokeh/release/bokeh-3.1.0.min.js"></script>\n    <script type="text/javascript">\n        Bokeh.set_log_level("info");\n    </script>\n  </head>\n  <body>\n    <div id="97c71d76-1b81-46b5-a564-e7c89f214688" data-root-id="p1125" style="display: contents;"></div>\n  \n    <script type="application/json" id="p1380">\n      {"dfece586-8d24-441a-a691-cf5919512272":{"version":"3.1.0","title":"Bokeh Application","defs":[],"roots":[{"type":"object","name":"Row","id":"p1125","attributes":{"children":[{"type":"object","name":"Figure","id":"p1005","attributes":{"width":800,"height":800,"x_range":{"type":"object","name":"FactorRange","id":"p1015","attributes":{"factors":["No

In [ ]:
# generate Bokeh plot components
script, div = components(layout)
div

In [ ]:
script